In [1]:
import torch

In [2]:
x = torch.tensor([1., 2.], requires_grad = True).view(-1,1)
t = torch.tensor([0.1, 0.2], requires_grad = True).view(-1,1)
print('x: ',x)
print('t: ',t)

x:  tensor([[1.],
        [2.]], grad_fn=<ViewBackward0>)
t:  tensor([[0.1000],
        [0.2000]], grad_fn=<ViewBackward0>)


**Class to define the neural network**

In [3]:
class Model(torch.nn.Module):
    def __init__(self) -> None:
        super(Model,self).__init__()
        self.layer01 = torch.nn.Linear(2,1)
    
    def forward(self,x,t):
        inputs = torch.cat([x,t], axis=1)
        output = torch.sigmoid(self.layer01(inputs))
        return output

In [4]:
model = Model()
model.state_dict()

OrderedDict([('layer01.weight', tensor([[0.6240, 0.0947]])),
             ('layer01.bias', tensor([-0.3125]))])

$$u(x,t;\theta) = \sigma(ax + bt + c)$$

In [5]:
u = model(x,t)

**First derivative w/r $x$**
$$u_{x}(x,t;\theta) = \sigma'(ax + bt + c)\frac{\partial}{\partial x}(ax + bt + c) = \sigma(ax + bt + c)(1 - \sigma(ax + bt + c))a$$

In [6]:
u_x  = torch.autograd.grad(outputs=u, 
                           inputs=x,
                           create_graph=True,
                           # Not sure when we may need 'retain_graph', maybe between epochs
                           #retain_graph=True,
                           grad_outputs=torch.ones_like(u)
                           )[0]
print(u_x)

tensor([[0.1521],
        [0.1252]], grad_fn=<SliceBackward0>)


In [7]:
# check
a = model.state_dict()['layer01.weight'][0][0]
b = model.state_dict()['layer01.weight'][0][1]
c = model.state_dict()['layer01.bias'][0]
print(torch.sigmoid(a*x + b*t + c)*(1 - torch.sigmoid(a*x + b*t + c))*a)

tensor([[0.1521],
        [0.1252]], grad_fn=<MulBackward0>)


**Second derivative w/r $x$**
$$u_{x}(x,t;\theta) = \sigma(ax + bt + c)(1 - \sigma(ax + bt + c))a$$
$$u_{xx}(x; \theta) = \sigma(ax + bt + c)(1 - \sigma(ax + bt + c))a^2 - \sigma(ax + bt + c)^2(1 - \sigma(ax + bt + c))a^2$$

In [8]:
u_xx = torch.autograd.grad(outputs=u_x, 
                           inputs=x,
                           create_graph=True,
                           grad_outputs=torch.ones_like(u_x)
                           )[0]
print(u_xx)

tensor([[-0.0151],
        [-0.0347]], grad_fn=<SliceBackward0>)


In [9]:
# check!
p1 = torch.sigmoid(a*x + b*t + c)*(1 - torch.sigmoid(a*x + b*t + c))**2*a**2
p2 = - torch.sigmoid(a*x + b*t + c)**2*(1 - torch.sigmoid(a*x + b*t + c))*a**2
print(p1 + p2)

tensor([[-0.0151],
        [-0.0347]], grad_fn=<AddBackward0>)


**Mixed derivatives w/r $x$ and $t$**
$$u_{x}(x,t;\theta) = \sigma(ax + bt + c)(1 - \sigma(ax + bt + c))a$$
$$u_{xt}(x; \theta) = \sigma(ax + bt + c)(1 - \sigma(ax + bt + c))ab - \sigma(ax + bt + c)^2(1 - \sigma(ax + bt + c))ab$$

In [10]:
u_xt = torch.autograd.grad(outputs=u_x, 
                           inputs=t,
                           grad_outputs=torch.ones_like(u_x)
                           )[0]
print(u_xt)

tensor([[-0.0023],
        [-0.0053]])


In [11]:
# check!
p1 = torch.sigmoid(a*x + b*t + c)*(1 - torch.sigmoid(a*x + b*t + c))**2*a*b
p2 = - torch.sigmoid(a*x + b*t + c)**2*(1 - torch.sigmoid(a*x + b*t + c))*a*b
print(p1 + p2)

tensor([[-0.0023],
        [-0.0053]], grad_fn=<AddBackward0>)
